In [1]:
import matplotlib.pyplot as plt
from coffea.util import load
from pocket_coffea.utils.plot_utils import Shape
from IPython.display import display, HTML
import pickle
import numpy as np
import os
import awkward as ak
from omegaconf import OmegaConf
import math
import pyarrow as pa

In [53]:
output = ak.from_parquet("/eos/user/r/rmccarth/ttHbb/chunks/TTToSemiLeptonic__2018/baseline/nominal/fcd64a3c-f13f-11eb-96b7-79bce183beef_#2FEvents#3B1_0-49616.parquet")
output2 = ak.from_parquet("/eos/user/r/rmccarth/ttHbb/chunks/TTToSemiLeptonic__2018/baseline/nominal/fcd64a3c-f13f-11eb-96b7-79bce183beef_#2FEvents#3B1_49616-99232.parquet")
output3 = ak.from_parquet("merge.parquet")

In [54]:
print(type(output))
print(output.fields)
print(len(output['JetGood_pt']))
print(len(output2['JetGood_pt']))
print(len(output3['JetGood_pt']))

<class 'awkward.highlevel.Array'>
['weight_nominal', 'weight_pileupUp', 'weight_pileupDown', 'weight_sf_ele_recoUp', 'weight_sf_ele_recoDown', 'weight_sf_ele_idUp', 'weight_sf_ele_idDown', 'weight_sf_mu_idUp', 'weight_sf_mu_idDown', 'weight_sf_mu_isoUp', 'weight_sf_mu_isoDown', 'weight_sf_jet_puIdUp', 'weight_sf_jet_puIdDown', 'weight_sf_btag_hfUp', 'weight_sf_btag_hfDown', 'weight_sf_btag_lfUp', 'weight_sf_btag_lfDown', 'weight_sf_btag_hfstats1Up', 'weight_sf_btag_hfstats1Down', 'weight_sf_btag_hfstats2Up', 'weight_sf_btag_hfstats2Down', 'weight_sf_btag_lfstats1Up', 'weight_sf_btag_lfstats1Down', 'weight_sf_btag_lfstats2Up', 'weight_sf_btag_lfstats2Down', 'weight_sf_btag_cferr1Up', 'weight_sf_btag_cferr1Down', 'weight_sf_btag_cferr2Up', 'weight_sf_btag_cferr2Down', 'JetGood_eta', 'JetGood_pt', 'JetGood_phi', 'JetGood_btagDeepFlavB', 'JetGood_genJetIdx', 'FatJetGood_eta', 'FatJetGood_pt', 'FatJetGood_phi', 'FatJetGood_mass', 'FatJetGood_msoftdrop', 'FatJetGood_tau1', 'FatJetGood_tau2',

In [4]:
def deltaR(phis,etas):
    leftPhi,rightPhi = ak.unzip(phis)
    leftEta,rightEta = ak.unzip(etas)
    return np.sqrt(pow(leftEta-rightEta,2)+pow(leftPhi-rightPhi,2))

samples=("ttHTobb",
         "ttHToNonbb",
         "TTbbDiLeptonic_Powheg",
         "TTbbSemiLeptonic_Powheg",
         "TTbbHadronic_Powheg",
         "THW",
         "TTToSemiLeptonic",
         "TTTo2L2Nu",
         "TTToHadronic",
         "TTZToQQ",
         "TTZtoLLNuNu",
         "TTWJetsToQQ",
         "TTWJetsToLNu",
         "TTGJets",
         "WW",
         "ZZ",
         "WZ",
         "WJetsToLNu_HT-200To400",
         "WJetsToLNu_HT-400To600",
         "WJetsToLNu_HT-600To800",
         "WJetsToLNu_HT-800To1200",
         "WJetsToLNu_HT-1200To2500",
         "WJetsToLNu_HT-2500ToInf",
         "WJetsToQQ_HT400to600_v7",
         "WJetsToQQ_HT600to800_v7",
         "WJetsToQQ_HT800toInf_v7",
         "ZJetsToQQ_HT400to600_v7",
         "ZJetsToQQ_HT600to800_v7",
         "ZJetsToQQ_HT800toInf_v7",
         "DYJetsToLL_M-50_v7",
         "ST_s-channel_4f_leptonDecays_v7",
         "ST_t-channel_top_4f_InclusiveDecays_v7",
         "ST_t-channel_antitop_4f_InclusiveDecays_v7",
         "ST_tW_top_5f_inclusiveDecays_v7",
         "ST_tW_antitop_5f_inclusiveDecays_v7",
         "QCD_HT500to700_v7",
         "QCD_HT700to1000_v7",
         "QCD_HT1000to1500_v7",
         "QCD_HT1500to2000_v7",
         "QCD_HT2000toInf_v7"
        )

years = ["2018"]
cats = ["baseline","ee","emu","mumu"]
variations = ["nominal"]
filepath = "/eos/user/r/rmccarth/ttHbb/chunks/"
#filepath="/eos/user/r/rmccarth/ttHbb/chunks/"
outputpath = "/eos/user/r/rmccarth/ttHbb/mergedFiles/"

In [6]:
for sample in ["ZZ__2018","WJetsToLNu_HT-200To400__2018","WJetsToLNu_HT-2500ToInf__2018","ST_tW_antitop_5f_inclusiveDecays_v7__2018","QCD_HT1500to2000_v7__2018","QCD_HT2000toInf_v7__2018","TTToSemiLeptonic__2018"]:
    for cat in cats:
        for variation in variations:
            path = filepath+sample+"/"+cat+"/"+variation
            outpath = outputpath+sample+"/"+cat+"/"+variation
            if(os.path.isdir(outpath)):
                continue
            ak.to_parquet.dataset(path)
            output = ak.from_parquet(path)

            output['FatJetGood_tau21'] = np.divide(output['FatJetGood_tau2'],output['FatJetGood_tau1'])
            output['FatJetGood_tau32'] = np.divide(output['FatJetGood_tau3'],output['FatJetGood_tau2'])
            output['FatJetGood_tau31'] = np.divide(output['FatJetGood_tau3'],output['FatJetGood_tau1'])
            output['FatJetGood_tau43'] = np.divide(output['FatJetGood_tau4'],output['FatJetGood_tau3'])
                
            output['BBFatJetGoodL_tau21'] = np.divide(output['BBFatJetGoodL_tau2'],output['BBFatJetGoodL_tau1'])
            output['BBFatJetGoodL_tau32'] = np.divide(output['BBFatJetGoodL_tau3'],output['BBFatJetGoodL_tau2'])
            output['BBFatJetGoodL_tau31'] = np.divide(output['BBFatJetGoodL_tau3'],output['BBFatJetGoodL_tau1'])
            output['BBFatJetGoodL_tau43'] = np.divide(output['BBFatJetGoodL_tau4'],output['BBFatJetGoodL_tau3'])
                
            output['BBFatJetGoodM_tau21'] = np.divide(output['BBFatJetGoodM_tau2'],output['BBFatJetGoodM_tau1'])
            output['BBFatJetGoodM_tau32'] = np.divide(output['BBFatJetGoodM_tau3'],output['BBFatJetGoodM_tau2'])
            output['BBFatJetGoodM_tau31'] = np.divide(output['BBFatJetGoodM_tau3'],output['BBFatJetGoodM_tau1'])
            output['BBFatJetGoodM_tau43'] = np.divide(output['BBFatJetGoodM_tau4'],output['BBFatJetGoodM_tau3'])

            output['BBFatJetGoodT_tau21'] = np.divide(output['BBFatJetGoodT_tau2'],output['BBFatJetGoodT_tau1'])
            output['BBFatJetGoodT_tau32'] = np.divide(output['BBFatJetGoodT_tau3'],output['BBFatJetGoodT_tau2'])
            output['BBFatJetGoodT_tau31'] = np.divide(output['BBFatJetGoodT_tau3'],output['BBFatJetGoodT_tau1'])
            output['BBFatJetGoodT_tau43'] = np.divide(output['BBFatJetGoodT_tau4'],output['BBFatJetGoodT_tau3'])

            jetLeptonEta = ak.cartesian([output["LeptonGood_eta"],output["JetGood_eta"]],axis=1)
            jetLeptonPhi = ak.cartesian([output["LeptonGood_phi"],output["JetGood_phi"]],axis=1)
            jetFatJetEta = ak.cartesian([output["FatJetGood_eta"],output["JetGood_eta"]],axis=1)
            jetFatJetPhi = ak.cartesian([output["FatJetGood_phi"],output["JetGood_phi"]],axis=1)
            fatjetLeptonEta = ak.cartesian([output["LeptonGood_eta"],output["FatJetGood_eta"]],axis=1)
            fatjetLeptonPhi = ak.cartesian([output["LeptonGood_phi"],output["FatJetGood_phi"]],axis=1)
            leptonsPhi = ak.to_list(ak.combinations(output["LeptonGood_phi"],2))
            leptonsEta = ak.to_list(ak.combinations(output["LeptonGood_eta"],2))
            jetPhi = ak.to_list(ak.combinations(output["JetGood_phi"],2))
            jetEta = ak.to_list(ak.combinations(output["JetGood_eta"],2))

            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0):
                leptonsDeltaR = deltaR(leptonsPhi,leptonsEta)
            else:
                leptonsDeltaR = []
            output['LeptonGoodCombinations_deltaR'] = leptonsDeltaR

            if(len(jetPhi)!=0 and len(jetEta)!=0):
                jetDeltaR = deltaR(jetPhi,jetEta)
            else:
                jetDeltaR = []
            output['JetGoodCombinations_deltaR'] = jetDeltaR

            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0 and len(jetPhi)!=0 and len(jetEta)!=0):
                jetLeptonDeltaR = deltaR(jetLeptonPhi,jetLeptonEta)
            else:
                jetLeptonDeltaR = []
            output['JetLeptonGoodCombinations_deltaR'] = jetLeptonDeltaR

            if(len(fatjetLeptonPhi)!=0 and len(fatjetLeptonEta)!=0):
                fatjetLeptonDeltaR = deltaR(fatjetLeptonPhi,fatjetLeptonEta)
            else:
                fatjetLeptonDeltaR = []
            output['FatJetLeptonGoodCombinations_deltaR'] = fatjetLeptonDeltaR

            if(len(jetFatJetPhi)!=0 and len(jetFatJetEta)!= 0):
                jetFatJetDeltaR = deltaR(jetFatJetPhi,jetFatJetEta)
            else:
                jetFatJetDeltaR = []
            output['JetFatJetGoodCombinations_deltaR'] = jetFatJetDeltaR
            os.makedirs(outpath)
            ak.to_parquet(output,f"{outpath}/merge.parquet")

In [8]:
os.path.isdir("/eos/user/r/rmccarth/ttHbb/mergedFiles/DATA_EGamma_2018_EraA/baseline/nominalergw")

False